In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

from __future__ import print_function

# 11.1 Association analysis

연관분석 

* Brute-force solutions : 무식하게 전부다 해보는

데이터 간의 관계 : 연관분석, 연관규칙 


# 11.2 The Apriori principle

* sparse matrix : 인스턴스에 존재 하는 아이템만 체크한 matrix

- frequent : 공통적으로 함께 발생한 아이템 목록 (지지도와 신뢰도를 사용)
- support : frequent item set 아이템이 전체집합에 중 포함된 확률 <br>
    (5개 집합중 4개에 속함 4/5)
- confidence : association rules 두개의 연관규칙이 얼마나 강한지 <br>
    (기저기가 있으면 두유가 있을 확률이 높다.(반대가 늘 참은 아님))
- transaction : 거래 목록 (사용자마다의 구매목록)

아이템이 하나씩 추가될때마다 조합에 대한 support, confidence 계산량이 너무 큼 

* Apriori principle : 관심있는 아이템의 집합만 고려 관심없는 조합을 가지치기함 
      특정 아이템 집합이 자주나오면 속해있는 부분집합도 자주 나온다. 
        예) 013 자주나오면 01, 03, 13 도 자주 나올것이다. 
      자주 나오지 않은 아이템은 합집합도 자주나오지 않을것이다.


# 11.3 Finding frequent itemsets with the Apriori algorithm


빈발 아이템 집합을 먼저 찾아야만 연관규칙을 찾을 수 있다.
입력 : 최소지지도와 데이터 집합이 필요 

## 11.3.1 Generating candidate itemsets

* 의사코드 
        for 데이터 집합 tran 에 있는 각각의 트랜잭션 반복 
        for 각 후보 아이템 집합 can 을 반복 
            can 이 tran 의 부분집합인지 확인
            부분집합이면 can 의 계수를 증가 
        for 후보 아이템 집합을 반복
            해당 아이템의 지지도가 최소이면 이 아이템을 저장
        빈발 아이템 목록 반환


In [59]:
import numpy as np
import apriori

In [16]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [17]:
# 1, 2, .. 아이템 하나별로 만듬 
def createC1(dataSet):
    C1 = []
    # 1,3,4 트랜젝션 
    for transaction in dataSet:
        
        for item in transaction:
            # c1 [[1],[1,2]] 에 [1,2] 가 있는지 
            if [item] not in C1 : # not [item] in C1
                C1.append([item]) # [] 중요!!
    
    C1.sort()
    return map(frozenset, C1)

#symmetric diff : 대칭차
# frozenset : immutable, hashable 한 집합 한번 만들고 변경 할 수 없다. (사전의 key 를 만들떄 사용)

In [18]:
a =[[1],[2], [1,2]]
map(frozenset, a)
map(set, a) # {1}집합

[frozenset({1}), frozenset({2}), frozenset({1, 2})]

[{1}, {2}, {1, 2}]

In [37]:
dataSet = loadDataSet()
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [87]:
# 지지도 구하기 
def scanD(D, Ck, minSupport):
    ssCnt = {}
    
    # 전체 trans 집합
    for tid in D:
        # frozenset 목록 
        for can in Ck:
            if can.issubset(tid):
                # 구매 횟수 print(ssCnt[can], itd)
                if not ssCnt.has_key(can): ssCnt[can]=1; 
                else: ssCnt[can] += 1
    print(ssCnt)
    # 아이템 개수 
    numItems = float(len(D))
    retList = []
    supportData = {}
    
    for key in ssCnt:
        # 4개 중에 2번 나오면 0.5 
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData


In [88]:
# 아이템 목록 집합으로 바꿔줌 
D = map(set, dataSet)
L1, suppData0 = scanD(D, C1, 0.5)
L1 # 최소 support 만족하는 c1 
suppData0

{frozenset([4]): 1, frozenset([5]): 3, frozenset([2]): 3, frozenset([3]): 3, frozenset([1]): 2}


[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})]

{frozenset({4}): 0.25,
 frozenset({5}): 0.75,
 frozenset({2}): 0.75,
 frozenset({3}): 0.75,
 frozenset({1}): 0.5}

In [91]:
a = map(frozenset,[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]])
type(a)
a[0].issubset([3])
a[0].issubset([1,3,4])

list

False

True

In [62]:
# 합집합 만들기 
# Lk : frozen{1}..4를 제외한 값, k : 2
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    
    for i in range(lenLk):
        # 1~4
        for j in range(i+1, lenLk): 
            # frozenset({1,2}), frozenset({1,3}) 의 부분 집합이 같은지 {1} 로 같음 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                # frozenset({1}), frozenset({3}) : frozenset({1,3})
                retList.append(Lk[i] | Lk[j]) #set union 합
                
    return retList

# Ck 로 부터 Lk 를 얻기 위해서 데이터 집합을 봄 
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = map(set, dataSet)
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1] # 중첩리스트 
    k = 2 # support 의 기준이 minSupport 이상인 값만 가지고 조합을 찾겠다. 
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData


In [63]:
L, suppData = apriori(dataSet)
L
suppData

# 지지도가 낮은 subset 은 버리고 관심있는 집합만 뽑음 

{frozenset([4]): 1, frozenset([5]): 3, frozenset([2]): 3, frozenset([3]): 3, frozenset([1]): 2}
{frozenset([1, 2]): 1, frozenset([1, 5]): 1, frozenset([3, 5]): 2, frozenset([2, 3]): 2, frozenset([2, 5]): 3, frozenset([1, 3]): 2}
{frozenset([2, 3, 5]): 2}
{}


[[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})],
 [frozenset({1, 3}), frozenset({2, 5}), frozenset({2, 3}), frozenset({3, 5})],
 [frozenset({2, 3, 5})],
 []]

{frozenset({5}): 0.75,
 frozenset({3}): 0.75,
 frozenset({2, 3, 5}): 0.5,
 frozenset({1, 2}): 0.25,
 frozenset({1, 5}): 0.25,
 frozenset({3, 5}): 0.5,
 frozenset({4}): 0.25,
 frozenset({2, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({1}): 0.5,
 frozenset({1, 3}): 0.5,
 frozenset({2}): 0.75}

# 11.4 Mining association rules from frequent item sets

* 선행조건 (antecedent) ->  결과 (consequent) 

* confidence (신뢰도) : support(P | H:합집합)/support(P)

    **012->3** 이 낮으면 02->12, 01->23 도 낮다. <br>
    **02->12, 01->23** 낮으면 0->123 도 낮다.

In [67]:
# 신뢰도 값을 가지고 규칙들의 리스트를 생성, 정렬해서 사용 
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

# 신뢰도 구하기 
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

# 연관 규칙을 만듬 
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)
            

In [68]:
rules = generateRules(L, suppData, minConf=0.7)
rules

frozenset([1]) --> frozenset([3]) conf: 1.0
frozenset([5]) --> frozenset([2]) conf: 1.0
frozenset([2]) --> frozenset([5]) conf: 1.0


[(frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0)]